In [1]:
import os
original_dir = os.getcwd()
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))
exec(open('setup/default.py').read())

In [3]:
menu_w_sold_out = pd.read_csv(os.getcwd()+'/data/preproc/main/menu_w_soldout.csv')

In [18]:
target_meal = 'lunch'
summary_df= menu_w_sold_out[menu_w_sold_out.meal_time == target_meal].groupby('day_of_week').agg(
    tot_day=('day', lambda x: x.nunique()),    # 고유한 date 개수 계산하여 n에 저장
    soldout_day=('day', lambda x: x[menu_w_sold_out.is_soldout==True].nunique())  # is_na=True 조건을 만족하는 고유한 date 개수 계산하여 n2에 저장
).reset_index()

summary_df['ratio'] = summary_df['soldout_day'] / summary_df['tot_day']  # n2/n 비율 계산
print(summary_df.sort_values(by='ratio',ascending=False))

target_meal = 'dinner'
summary_df= menu_w_sold_out[menu_w_sold_out.meal_time == target_meal].groupby('day_of_week').agg(
    tot_day=('day', lambda x: x.nunique()),    # 고유한 date 개수 계산하여 n에 저장
    soldout_day=('day', lambda x: x[menu_w_sold_out.is_soldout==True].nunique())  # is_na=True 조건을 만족하는 고유한 date 개수 계산하여 n2에 저장
).reset_index()

summary_df['ratio'] = summary_df['soldout_day'] / summary_df['tot_day']  # n2/n 비율 계산
print(summary_df.sort_values(by='ratio',ascending=False))

  day_of_week  tot_day  soldout_day     ratio
3     Tuesday       94           60  0.638298
0      Friday       95           59  0.621053
2    Thursday       89           52  0.584270
1      Monday       86           48  0.558140
4   Wednesday       96           48  0.500000
  day_of_week  tot_day  soldout_day     ratio
3     Tuesday       94           53  0.563830
1      Monday       85           45  0.529412
2    Thursday       97           47  0.484536
4   Wednesday       95           46  0.484211
0      Friday       96            0  0.000000
